In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
import gc

In [51]:
feature = pd.read_csv('/Users/wasedafuuta/Desktop/keiba/netkeiba_data/data20181209/feature.csv')
payoff = pd.read_csv('/Users/wasedafuuta/Desktop/keiba/netkeiba_data/data20181209/payoff.csv')
race_info = pd.read_csv('/Users/wasedafuuta/Desktop/keiba/netkeiba_data/data20181209/race_info.csv')
race_info = race_info.rename(columns={'id': 'race_id'})
race_result = pd.read_csv('/Users/wasedafuuta/Desktop/keiba/netkeiba_data/data20181209/race_result.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
train_df = feature

race_result=race_result.drop({'order_of_finish','horse_number',
                              'sex','age','jockey_id','odds','remark','trainer_id',
                              'owner_id','horse_weight'},axis=1)
train_df = pd.merge(train_df, race_result, on=['race_id', 'horse_id'],how='left')

race_info=race_info.drop({'race_name','surface','distance',
                          'weather','surface_state','place_detail','race_class'},axis=1)
train_df=pd.merge(train_df, race_info , on='race_id' , how='left')

train_df=train_df.drop({'avgsr4','avgWin4','disavesr','dsl',
                'surfaceScore','jAvgWin4','preSRa','surfaceScore'},axis=1)


In [53]:
train_df.tail()

,race_id,horse_number,grade,order_of_finish,horse_id,jockey_id,trainer_id,age,dhweight,disRoc,distance,enterTimes,eps,hweight,jwinper,odds,owinper,sex,surface,twinper,weather,weight,winRun,jEps,preOOF,pre2OOF,month,ridingStrongJockey,runningStyle,preLateStart,preLastPhase,lateStartPer,course,placeCode,headCount,preHeadCount,surfaceChanged,gradeChanged,preMargin,femaleOnly,frame_number,basis_weight,finishing_time,length,speed_figure,pass,last_phase,popularity,stable,earning_money,race_start,race_number,surface_score,date
504745,34990,18,2,14.0,2013103969,1114,356,5.0,6.0,NaN,1200.0,0.0,NaN,470.0,0.00000,348.2,NaN,セ,芝,0.071429,晴,53.0,NaN,25.000000,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0,8,53.0,1:10.0,クビ,NaN,13-12,35.1,18.0,西,NaN,14:25,9,NaN,2018-10-20
504746,34990,11,2,15.0,2014103884,894,427,4.0,3.0,NaN,1200.0,0.0,NaN,538.0,0.00000,256.1,0.0,牡,芝,0.000000,晴,53.0,NaN,18.928571,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0,6,53.0,1:10.0,クビ,NaN,17-17,34.9,17.0,西,NaN,14:25,9,NaN,2018-10-20
504747,34990,12,2,16.0,2015103673,1156,1028,3.0,8.0,NaN,1200.0,0.0,NaN,430.0,0.00000,22.1,0.0,牝,芝,0.066667,晴,51.0,NaN,35.111111,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0,6,51.0,1:10.1,クビ,NaN,1-1,36.0,11.0,西,NaN,14:25,9,NaN,2018-10-20
504748,34990,4,2,17.0,2013103801,1037,1078,5.0,12.0,NaN,1200.0,0.0,NaN,488.0,0.00000,50.4,0.0,牡,芝,0.181818,晴,56.0,NaN,68.304167,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0,2,56.0,1:10.2,3/4,NaN,17-17,35.0,15.0,西,NaN,14:25,9,NaN,2018-10-20
504749,34990,7,2,18.0,2011105974,1018,1087,7.0,4.0,NaN,1200.0,0.0,NaN,528.0,0.06383,16.8,NaN,牝,芝,0.083333,晴,53.0,NaN,112.553191,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0,4,53.0,1:10.7,3,NaN,3-3,36.3,7.0,西,NaN,14:25,9,NaN,2018-10-20


In [54]:
def grade_0_fukusho(df):
    if df['grade']==0 and df['order_of_finish']<=3 : return 1
    else: return 0
def grade_1_fukusho(df):
    if df['grade']==1 and df['order_of_finish']<=3 : return 1
    else: return 0
def grade_2_fukusho(df):
    if df['grade']==2 and df['order_of_finish']<=3 : return 1
    else: return 0
def grade_3_fukusho(df):
    if df['grade']==3 and df['order_of_finish']<=3 : return 1
    else: return 0
def grade_4_fukusho(df):
    if df['grade']==4 and df['order_of_finish']<=3 : return 1
    else: return 0
def grade_5_fukusho(df):
    if df['grade']==5 and df['order_of_finish']<=3 : return 1
    else: return 0

In [55]:
def strFloat_to_strInt(x):
    return str(x).replace('.0','')

In [56]:
def passOrd(x):
    return int(str(x).split('-')[-1])

In [57]:
def pre_pre2_RunStyle(race_id, horse_id, data):
    eachHorse=data[data['horse_id']==horse_id].sort_values('date')
    eachHorse=eachHorse.reset_index()
    n = eachHorse[eachHorse['race_id']==race_id].index[0]
    if n==0:
        pre,pre2= -1, -1
    elif n==1:
        pre,pre2= eachHorse.at[n-1, 'runningStyle'], -1
    else:
        pre,pre2=eachHorse.at[n-1, 'runningStyle'], eachHorse.at[n-2, 'runningStyle']
    del eachHorse
    gc.collect()
    return pre, pre2

In [58]:
def dammy(data , columns):
    for col in columns:
        dammy_df=pd.get_dummies(data[[col]].astype(object))
        data=pd.concat([data, dammy_df], axis=1)
        data=data.drop(col,axis=1)
        del dammy_df
        gc.collect()
    return data

In [59]:
def fillna_0(data, columns):
    for col in columns:
        data[col] = data[col].fillna(0)
    return data

In [60]:
def fillna_mean(data, columns):
    for col in columns:
        data[col] = data[col].fillna(data[col].mean())
    return data

In [63]:
def PreProcess(data):
    #脚質を計算、追加、元の脚質情報は削除　（通過順位−１）／（頭数−１）
    data['pass'] = data['pass'].fillna(-1)
    data['pass'] = data['pass'].map(passOrd)
    data['runningStyle']=(data['pass']-1)/(data['headCount']-1)
    
    #preOOF,pre2OOFの数字を整数に
    data['preOOF']= data['preOOF'].map(strFloat_to_strInt)
    data['pre2OOF']= data['pre2OOF'].map(strFloat_to_strInt)
    
    #fillna_0
    fillna_0_cols = ['ridingStrongJockey', 'eps', 'jwinper','owinper','twinper','winRun', 'jEps','runningStyle','lateStartPer']
    data = fillna_0(data, fillna_0_cols)

    #fillna_mean
    fillna_mean_cols = ['dhweight', 'hweight', 'preLastPhase']
    data = fillna_mean(data ,fillna_mean_cols)
    
    #odds 無限でfillna
    data['odds']= data['odds'].fillna(np.inf)
    
    #disRoc
    data=data.drop('disRoc',axis=1)
    
    #j-hweight　　（騎手＋斤量）／馬の体重
    data['j-hweight']=(data['basis_weight']+data['weight']) / data['hweight']
    
    #支持率
    data['supPer'] = 0.788/(data['odds']-0.1)
    data=data.drop('odds',axis=1)
    
    #馬の１着数
    data['winRunTimes']=data['winRun']*data['enterTimes']

    #そのレースの脚質の平均
    runStyle=pd.DataFrame(data.groupby('race_id',as_index=False)[['runningStyle']].mean())
    runStyle = runStyle.rename(columns={'runningStyle': 'all_runnnigStyle'})
    data = pd.merge(data, runStyle , on='race_id' ,how='left')

    #前走、前々走の脚質
    for n in range(len(data)):
        n_race_id=data['race_id'].loc[n]
        n_horse_id=data['horse_id'].loc[n]
        pre , pre2 = pre_pre2_RunStyle(n_race_id, n_horse_id ,data)
        data.at[n,'preRunningStyle'] = pre
        data.at[n,'pre2RunningStyle'] = pre2
        
    #2着数、３着数
    data['order_was_2']=0
    data['order_was_3']=0
    for horse_id in data.horse_id.unique():
        eachHorse=data[data['horse_id']==horse_id].sort_values('date').reset_index()
        first_race_idx = eachHorse.loc[0,'index']
        for n in range(len(eachHorse)-1):
            this_race_idx=eachHorse.loc[n,'index']
            next_race_idx = eachHorse.loc[n+1,'index']
            if eachHorse.loc[n,'order_of_finish']==2:
                data.at[next_race_idx, 'order_was_2'] =data.at[this_race_idx,'order_was_2'] + 1
                data.at[next_race_idx, 'order_was_3'] =  data.at[this_race_idx, 'order_was_3']
            elif eachHorse.loc[n,'order_of_finish']==3:
                data.at[next_race_idx, 'order_was_2'] =data.at[this_race_idx,'order_was_2']
                data.at[next_race_idx, 'order_was_3'] =  data.at[this_race_idx,'order_was_3'] + 1
            else :
                data.at[next_race_idx, 'order_was_2'] =data.at[this_race_idx,'order_was_2'] 
                data.at[next_race_idx, 'order_was_3'] =data.at[this_race_idx,'order_was_3']
        del eachHorse
        gc.collect()
        
    #1~3着でない回数
    data['not123times'] = data['enterTimes']-data['winRunTimes']-data['order_was_2']-data['order_was_3']
    
    #距離変化量 changedist
    for horse_id in data.horse_id.unique():
        eachHorse=data[data['horse_id']==horse_id].sort_values('date').reset_index()
        first_race_idx = eachHorse.loc[0,'index']
        data.at[first_race_idx,'changeDist']=0
        for n in range(1, len(eachHorse)):
            before_race_idx = eachHorse.loc[n-1,'index']
            this_race_idx = eachHorse.loc[n,'index']
            data.at[this_race_idx,'changeDist']=data.at[this_race_idx,'distance']-data.at[before_race_idx,'distance']
        del eachHorse
        gc.collect()
        
    #今回の３ハロン順位結果
    for race_id in data.race_id.unique():
        eachRace = data[data['race_id']==race_id].sort_values('last_phase').reset_index()
        for n in range(len(eachRace)) :
            idx = eachRace.loc[n,'index']
            data.at[idx, 'last_phase_order'] =  n+1
        del eachRace
        gc.collect()
    #前回の３ハロン順位
    for horse_id in data.horse_id.unique():
        eachHorse=data[data['horse_id']==horse_id].sort_values('date').reset_index()
        first_race_idx = eachHorse.loc[0,'index']
        data.at[first_race_idx,'pre_last_phase_order']=0
        for n in range(1, len(eachHorse)):
            before_race_idx = eachHorse.loc[n-1,'index']
            this_race_idx = eachHorse.loc[n,'index']
            data.at[this_race_idx,'pre_last_phase_order']=data.at[before_race_idx,'last_phase_order']
        del eachHorse
        gc.collect()
    
    #ランク別の複勝数
    data['grade_0_fukusho'] = data.apply(grade_0_fukusho, axis=1)
    data['grade_1_fukusho'] = data.apply(grade_1_fukusho, axis=1)
    data['grade_2_fukusho'] = data.apply(grade_2_fukusho, axis=1)
    data['grade_3_fukusho'] = data.apply(grade_3_fukusho, axis=1)
    data['grade_4_fukusho'] = data.apply(grade_4_fukusho, axis=1)
    data['grade_5_fukusho'] = data.apply(grade_5_fukusho, axis=1)
    data['grade_0_fukusho_sum'] = 0
    data['grade_1_fukusho_sum'] = 0
    data['grade_2_fukusho_sum'] = 0
    data['grade_3_fukusho_sum'] = 0
    data['grade_4_fukusho_sum'] = 0
    data['grade_5_fukusho_sum'] = 0
    for horse_id in data.horse_id.unique():
        eachHorse=data[data['horse_id']==horse_id].sort_values('date').reset_index()
        first_race_idx = eachHorse.loc[0,'index']
        for n in range(len(eachHorse)-1):
            this_race_idx=eachHorse.loc[n,'index']
            next_race_idx = eachHorse.loc[n+1,'index']
            data.at[next_race_idx, 'grade_0_fukusho_sum'] = data.at[this_race_idx, 'grade_0_fukusho_sum'] + data.at[this_race_idx, 'grade_0_fukusho']
            data.at[next_race_idx, 'grade_1_fukusho_sum'] = data.at[this_race_idx, 'grade_1_fukusho_sum'] + data.at[this_race_idx, 'grade_1_fukusho']
            data.at[next_race_idx, 'grade_2_fukusho_sum'] = data.at[this_race_idx, 'grade_2_fukusho_sum'] + data.at[this_race_idx, 'grade_2_fukusho']
            data.at[next_race_idx, 'grade_3_fukusho_sum'] = data.at[this_race_idx, 'grade_3_fukusho_sum'] + data.at[this_race_idx, 'grade_3_fukusho']
            data.at[next_race_idx, 'grade_4_fukusho_sum'] = data.at[this_race_idx, 'grade_4_fukusho_sum'] + data.at[this_race_idx, 'grade_4_fukusho']
            data.at[next_race_idx, 'grade_5_fukusho_sum'] = data.at[this_race_idx, 'grade_5_fukusho_sum'] + data.at[this_race_idx, 'grade_5_fukusho']
        del eachHorse
        gc.collect()
    #そのグレードより高いグレードでの複勝数
    data['this_grade_fukusho_sum']  = 0
    for n in range(len(data)):
        if data.at[n,'grade'] == 0:
            this_grade_fukusho_sum = data.at[n, 'grade_0_fukusho_sum']
        elif data.at[n,'grade'] == 1:
            this_grade_fukusho_sum = data.at[n, 'grade_0_fukusho_sum'] + data.at[n, 'grade_1_fukusho_sum']
        elif data.at[n,'grade'] == 2:
            this_grade_fukusho_sum = data.at[n, 'grade_0_fukusho_sum'] + data.at[n, 'grade_1_fukusho_sum'] +data.at[n,'grade_2_fukusho_sum']
        elif data.at[n,'grade'] == 3:
            this_grade_fukusho_sum = data.at[n, 'grade_0_fukusho_sum'] + data.at[n, 'grade_1_fukusho_sum'] +data.at[n, 'grade_2_fukusho_sum'] + data.at[n, 'grade_3_fukusho_sum']
        elif data.at[n,'grade'] == 4:
            this_grade_fukusho_sum = data.at[n, 'grade_0_fukusho_sum'] + data.at[n, 'grade_1_fukusho_sum'] +data.at[n,'grade_2_fukusho_sum'] + data.at[n, 'grade_3_fukusho_sum'] + data.at[n, 'grade_4_fukusho_sum']
        elif data.at[n,'grade'] == 5:
            this_grade_fukusho_sum = data.at[n, 'grade_0_fukusho_sum'] + data.at[n, 'grade_1_fukusho_sum'] +data.at[n, 'grade_2_fukusho_sum'] + data.at[n, 'grade_3_fukusho_sum'] + data.at[n, 'grade_4_fukusho_sum'] + data.at[n, 'grade_5_fukusho_sum']
        data.at[n, 'this_grade_fukusho_sum'] = this_grade_fukusho_sum

        
    
    
        
    #ダミー変数化
    dammy_cols = ['horse_number','grade','sex','surface','weather','preOOF',
                     'pre2OOF' ,'month', 'ridingStrongJockey' , 'course' ,'placeCode' ,'headCount' ,
                     'preHeadCount' ,'surfaceChanged','gradeChanged','femaleOnly','popularity','stable']
    data = dammy(data, dammy_cols)
    
    return data

In [64]:
data_ = PreProcess(train_df)

In [ ]:
data_.to_csv('preProcessed_df.csv' , index=False)

In [13]:
data_.isnull().any(axis=0)

race_id                   False
order_of_finish            True
horse_id                  False
jockey_id                 False
trainer_id                False
age                       False
dhweight                  False
distance                  False
enterTimes                False
eps                       False
hweight                   False
jwinper                   False
owinper                   False
twinper                   False
weight                    False
winRun                    False
jEps                      False
runningStyle              False
preLateStart              False
preLastPhase              False
lateStartPer              False
preMargin                  True
frame_number              False
basis_weight              False
finishing_time             True
length                     True
speed_figure               True
pass                      False
last_phase                 True
earning_money              True
race_start                False
race_num

In [170]:
df = data_

In [66]:
df.head()

,race_id,order_of_finish,horse_id,jockey_id,trainer_id,age,dhweight,distance,enterTimes,eps,hweight,jwinper,owinper,twinper,weight,winRun,jEps,runningStyle,preLateStart,preLastPhase,lateStartPer,preMargin,frame_number,basis_weight,finishing_time,length,speed_figure,pass,last_phase,earning_money,race_start,race_number,surface_score,date,j-hweight,supPer,winRunTimes,all_runnnigStyle,preRunningStyle,pre2RunningStyle,order_was_2,order_was_3,not123times,changeDist,pre_last_phase_order,grade_0_fukusho_sum,grade_1_fukusho_sum,grade_2_fukusho_sum,grade_3_fukusho_sum,grade_4_fukusho_sum,grade_5_fukusho_sum,this_grade_fukusho_sum,horse_number_1,horse_number_2,horse_number_3,horse_number_4,horse_number_5,horse_number_6,horse_number_7,horse_number_8,horse_number_9,horse_number_10,horse_number_11,horse_number_12,horse_number_13,horse_number_14,horse_number_15,horse_number_16,horse_number_17,horse_number_18,grade_0,grade_1,grade_2,grade_3,grade_4,grade_5,sex_セ,sex_牝,sex_牡,surface_ダ,surface_芝,weather_他,weather_小雨,weather_晴,weather_曇,weather_雨,weather_雪,preOOF_1,preOOF_10,preOOF_10(降),preOOF_11,preOOF_11(降),preOOF_12,preOOF_12(降),preOOF_13,preOOF_13(降),preOOF_14,preOOF_14(降),preOOF_15,preOOF_15(降),preOOF_16,preOOF_16(降),preOOF_17,preOOF_17(降),preOOF_18,preOOF_18(降),preOOF_2,preOOF_2(降),preOOF_3,preOOF_3(降),preOOF_4,preOOF_4(降),preOOF_5,preOOF_5(降),preOOF_6,preOOF_6(降),preOOF_7,preOOF_7(降),preOOF_8,preOOF_8(降),preOOF_9,preOOF_9(降),preOOF_nan,preOOF_中,preOOF_取,preOOF_失,preOOF_除,pre2OOF_1,pre2OOF_10,pre2OOF_10(降),pre2OOF_11,pre2OOF_11(降),pre2OOF_12,pre2OOF_12(降),pre2OOF_13,pre2OOF_13(降),pre2OOF_14,pre2OOF_14(降),pre2OOF_15,pre2OOF_15(降),pre2OOF_16,pre2OOF_16(降),pre2OOF_17,pre2OOF_17(降),pre2OOF_18,pre2OOF_18(降),pre2OOF_2,pre2OOF_2(降),pre2OOF_3,pre2OOF_3(降),pre2OOF_4,pre2OOF_4(降),pre2OOF_5,pre2OOF_5(降),pre2OOF_6,pre2OOF_6(降),pre2OOF_7,pre2OOF_7(降),pre2OOF_8,pre2OOF_8(降),pre2OOF_9,pre2OOF_9(降),pre2OOF_nan,pre2OOF_中,pre2OOF_取,pre2OOF_失,pre2OOF_除,month_1.0,month_2.0,month_3.0,month_4.0,month_5.0,month_6.0,month_7.0,month_8.0,month_9.0,month_10.0,month_11.0,month_12.0,ridingStrongJockey_0.0,ridingStrongJockey_1.0,course_他,course_右,course_外,course_左,course_直線,placeCode_中京,placeCode_中山,placeCode_京都,placeCode_函館,placeCode_小倉,placeCode_新潟,placeCode_札幌,placeCode_東京,placeCode_福島,placeCode_阪神,headCount_5.0,headCount_6.0,headCount_7.0,headCount_8.0,headCount_9.0,headCount_10.0,headCount_11.0,headCount_12.0,headCount_13.0,headCount_14.0,headCount_15.0,headCount_16.0,headCount_17.0,headCount_18.0,preHeadCount_5.0,preHeadCount_6.0,preHeadCount_7.0,preHeadCount_8.0,preHeadCount_9.0,preHeadCount_10.0,preHeadCount_11.0,preHeadCount_12.0,preHeadCount_13.0,preHeadCount_14.0,preHeadCount_15.0,preHeadCount_16.0,preHeadCount_17.0,preHeadCount_18.0,surfaceChanged_0.0,surfaceChanged_1.0,gradeChanged_-4.0,gradeChanged_-3.0,gradeChanged_-2.0,gradeChanged_-1.0,gradeChanged_0.0,gradeChanged_1.0,gradeChanged_2.0,gradeChanged_3.0,gradeChanged_4.0,gradeChanged_5.0,femaleOnly_0.0,femaleOnly_1.0,popularity_1.0,popularity_2.0,popularity_3.0,popularity_4.0,popularity_5.0,popularity_6.0,popularity_7.0,popularity_8.0,popularity_9.0,popularity_10.0,popularity_11.0,popularity_12.0,popularity_13.0,popularity_14.0,popularity_15.0,popularity_16.0,popularity_17.0,popularity_18.0,stable_地,stable_外,stable_東,stable_西
0,1,1.0,2006105662,1102,353,2.0,4.0,1200.0,0.0,0.0,452.0,0.0,0.0,0.0,54.0,0.0,0.0,0.000000,0.0,36.272991,0.0,NaN,5,54.0,1:10.2,NaN,NaN,1,36.3,500.0,10:00,1,NaN,2008-11-01,0.238938,0.114203,0.0,0.454167,-1.0,-1.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,2.0,2006105747,635,429,2.0,0

In [173]:
df=df.drop({'preMargin','finishing_time','length','speed_figure',
           'last_phase','race_start','surface_score',
            'earning_money','date','runningStyle','all_runnnigStyle','pass', 
            }, axis=1)


In [ ]:
'last_phase_order','grade_0_fukusho', 'grade_1_fukusho','grade_2_fukusho',
                      'grade_3_fukusho','grade_4_fukusho','grade_5_fukusho'

In [175]:
df.shape

(504750, 248)

In [174]:
df.isnull().sum()

race_id                      0
order_of_finish           4269
horse_id                     0
jockey_id                    0
trainer_id                   0
age                          0
dhweight                     0
distance                     0
enterTimes                   0
eps                          0
hweight                      0
jwinper                      0
owinper                      0
twinper                      0
weight                       0
winRun                       0
jEps                         0
preLateStart                 0
preLastPhase                 0
lateStartPer                 0
frame_number                 0
basis_weight                 0
race_number                  0
j-hweight                    0
supPer                       0
winRunTimes                  0
preRunningStyle              0
pre2RunningStyle             0
order_was_2                  0
order_was_3                  0
not123times                  0
changeDist                   0
pre_last

In [176]:
df=df.dropna(how='any')

In [177]:
df.shape

(500481, 248)

In [73]:
df.head()

,race_id,order_of_finish,horse_id,jockey_id,trainer_id,age,dhweight,distance,enterTimes,eps,hweight,jwinper,owinper,twinper,weight,winRun,jEps,preLateStart,preLastPhase,lateStartPer,frame_number,basis_weight,race_number,j-hweight,supPer,winRunTimes,preRunningStyle,pre2RunningStyle,order_was_2,order_was_3,not123times,changeDist,pre_last_phase_order,grade_0_fukusho_sum,grade_1_fukusho_sum,grade_2_fukusho_sum,grade_3_fukusho_sum,grade_4_fukusho_sum,grade_5_fukusho_sum,this_grade_fukusho_sum,horse_number_1,horse_number_2,horse_number_3,horse_number_4,horse_number_5,horse_number_6,horse_number_7,horse_number_8,horse_number_9,horse_number_10,horse_number_11,horse_number_12,horse_number_13,horse_number_14,horse_number_15,horse_number_16,horse_number_17,horse_number_18,grade_0,grade_1,grade_2,grade_3,grade_4,grade_5,sex_セ,sex_牝,sex_牡,surface_ダ,surface_芝,weather_他,weather_小雨,weather_晴,weather_曇,weather_雨,weather_雪,preOOF_1,preOOF_10,preOOF_10(降),preOOF_11,preOOF_11(降),preOOF_12,preOOF_12(降),preOOF_13,preOOF_13(降),preOOF_14,preOOF_14(降),preOOF_15,preOOF_15(降),preOOF_16,preOOF_16(降),preOOF_17,preOOF_17(降),preOOF_18,preOOF_18(降),preOOF_2,preOOF_2(降),preOOF_3,preOOF_3(降),preOOF_4,preOOF_4(降),preOOF_5,preOOF_5(降),preOOF_6,preOOF_6(降),preOOF_7,preOOF_7(降),preOOF_8,preOOF_8(降),preOOF_9,preOOF_9(降),preOOF_nan,preOOF_中,preOOF_取,preOOF_失,preOOF_除,pre2OOF_1,pre2OOF_10,pre2OOF_10(降),pre2OOF_11,pre2OOF_11(降),pre2OOF_12,pre2OOF_12(降),pre2OOF_13,pre2OOF_13(降),pre2OOF_14,pre2OOF_14(降),pre2OOF_15,pre2OOF_15(降),pre2OOF_16,pre2OOF_16(降),pre2OOF_17,pre2OOF_17(降),pre2OOF_18,pre2OOF_18(降),pre2OOF_2,pre2OOF_2(降),pre2OOF_3,pre2OOF_3(降),pre2OOF_4,pre2OOF_4(降),pre2OOF_5,pre2OOF_5(降),pre2OOF_6,pre2OOF_6(降),pre2OOF_7,pre2OOF_7(降),pre2OOF_8,pre2OOF_8(降),pre2OOF_9,pre2OOF_9(降),pre2OOF_nan,pre2OOF_中,pre2OOF_取,pre2OOF_失,pre2OOF_除,month_1.0,month_2.0,month_3.0,month_4.0,month_5.0,month_6.0,month_7.0,month_8.0,month_9.0,month_10.0,month_11.0,month_12.0,ridingStrongJockey_0.0,ridingStrongJockey_1.0,course_他,course_右,course_外,course_左,course_直線,placeCode_中京,placeCode_中山,placeCode_京都,placeCode_函館,placeCode_小倉,placeCode_新潟,placeCode_札幌,placeCode_東京,placeCode_福島,placeCode_阪神,headCount_5.0,headCount_6.0,headCount_7.0,headCount_8.0,headCount_9.0,headCount_10.0,headCount_11.0,headCount_12.0,headCount_13.0,headCount_14.0,headCount_15.0,headCount_16.0,headCount_17.0,headCount_18.0,preHeadCount_5.0,preHeadCount_6.0,preHeadCount_7.0,preHeadCount_8.0,preHeadCount_9.0,preHeadCount_10.0,preHeadCount_11.0,preHeadCount_12.0,preHeadCount_13.0,preHeadCount_14.0,preHeadCount_15.0,preHeadCount_16.0,preHeadCount_17.0,preHeadCount_18.0,surfaceChanged_0.0,surfaceChanged_1.0,gradeChanged_-4.0,gradeChanged_-3.0,gradeChanged_-2.0,gradeChanged_-1.0,gradeChanged_0.0,gradeChanged_1.0,gradeChanged_2.0,gradeChanged_3.0,gradeChanged_4.0,gradeChanged_5.0,femaleOnly_0.0,femaleOnly_1.0,popularity_1.0,popularity_2.0,popularity_3.0,popularity_4.0,popularity_5.0,popularity_6.0,popularity_7.0,popularity_8.0,popularity_9.0,popularity_10.0,popularity_11.0,popularity_12.0,popularity_13.0,popularity_14.0,popularity_15.0,popularity_16.0,popularity_17.0,popularity_18.0,stable_地,stable_外,stable_東,stable_西
0,1,1.0,2006105662,1102,353,2.0,4.0,1200.0,0.0,0.0,452.0,0.0,0.0,0.0,54.0,0.0,0.0,0.0,36.272991,0.0,5,54.0,1,0.238938,0.114203,0.0,-1.0,-1.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,2.0,2006105747,635,429,2.0,0.0,1200.0,0.0,0.0,440.0,0.0,0.0,0.0,54.0,0.0,0.0,0.0,36.272991,0.0,6,54.0,1,0.245455,0.197000,0.0,-1.0,-1.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,

In [74]:
df.to_csv('learn_df.csv' , index=False)

In [169]:
df = pd.read_csv('learn_df.csv')
df.shape

FileNotFoundError: File b'learn_df.csv' does not exist

In [75]:
for col in df.columns:
    if df[col].isnull().any()==True:
        print(col)

# 着順を１、０のバイナリ値に

In [179]:
def order(x):
    if x==1 or  x==2  or x==3:
        return 1
    else :
        return 0

In [180]:
df['order_of_finish']=df['order_of_finish'].map(order)

In [78]:
def Split(df):
    from sklearn.model_selection import train_test_split, GridSearchCV
    X,y =df.drop( ['order_of_finish', 'race_id','horse_id','jockey_id','trainer_id'],axis=1) , df['order_of_finish']
    #列名順に並び替え
    X=X.sort_index(axis=1, ascending=True)
    X_train ,X_test, y_train, y_test =  train_test_split(X, y , test_size=0.2 , random_state=0, stratify=y)
    return X_train ,X_test, y_train, y_test

def Standardize(X_train):
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    #fitで、平均値と標準偏差を推定
    sc.fit(X_train)
    #transformで、fitで推定した値を元に標準化
    X_train_std = sc.transform(X_train)
    X_test_std  = sc.transform(X_test)
    return X_train_std,X_test_std ,sc

In [79]:
def AUC(method ,X_test_std, y_test):
    from sklearn import metrics
    from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, roc_curve
    print(roc_auc_score(y_test, method.predict_proba(X_test_std)[:,1]))

In [80]:
def AUC_(method ,X_test_std, y_test):
    from sklearn import metrics
    from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, roc_curve
    print(roc_auc_score(y_test, method.predict(X_test_std)))

# レースごとに標準化

In [178]:
def standard_eachRace(df):
    #インデックスをリセット
    df = df.reset_index().drop('index',axis=1)
    #標準化
    standardize_col = list( set(list(df.columns.values)) - set(['race_id','horse_id','jockey_id','trainer_id','order_of_finish']) )
    for race_id in list(df['race_id'].unique()):
        eachRace = df[df['race_id']==race_id].reset_index()
        for col in standardize_col:
            ave = round(eachRace[col].mean(),4)
            std = round(eachRace[col].std(),4)
            for n in range(len(eachRace)) :
                idx = eachRace.loc[n,'index']
                if std ==0:
                    df.at[idx, col]  = 0
                else:
                    df.at[idx, col]  = round((df.at[idx, col] - ave )/std, 4)
        if race_id%1000 == 0:
            print(race_id)
    return df

In [ ]:
std_df = standard_eachRace(df)

1000
2000
3000
4000
5000
6000
7000
8000
9000


In [ ]:
df.head()

# 1~30000:学習、30001~:テストに分ける

In [117]:
def Split_by_race_id(df):
    df_train = df[df['race_id']<=30000]
    df_test = df[df['race_id']>30000]

    X_train  =  df_train.drop(['race_id','horse_id','jockey_id','trainer_id','order_of_finish'] ,axis=1) 
    y_train = df_train['order_of_finish']
    X_test = df_test.drop(['race_id','horse_id','jockey_id','trainer_id','order_of_finish'],axis=1) 
    y_test = df_test['order_of_finish']
    return X_train ,X_test, y_train, y_test

In [ ]:
std_df = std_df.sort_index(axis=1)
X_train_std , X_test_std,y_train, y_test =Split_by_race_id(std_df)

# LightGBM

In [ ]:
import lightgbm
from lightgbm import LGBMClassifier as lgb
clf = lgb(
            n_estimators=4000,
            learning_rate=0.03,
            num_leaves=30,
            colsample_bytree=.8,
            subsample=.9,
            max_depth=7,
            reg_alpha=.1,
            reg_lambda=.1,
            min_split_gain=.01,
            min_child_weight=2,
            silent=-1,
            verbose=-1,
        )

clf.fit(X_train_std, y_train )
AUC(clf,X_test_std, y_test)

In [438]:
X_train.head()

,race_id,horse_id,jockey_id,trainer_id,age,dhweight,distance,enterTimes,eps,hweight,jwinper,owinper,twinper,weight,winRun,jEps,preLateStart,preLastPhase,lateStartPer,frame_number,basis_weight,race_number,horse_number_1,horse_number_2,horse_number_3,horse_number_4,horse_number_5,horse_number_6,horse_number_7,horse_number_8,horse_number_9,horse_number_10,horse_number_11,horse_number_12,horse_number_13,horse_number_14,horse_number_15,horse_number_16,horse_number_17,horse_number_18,grade_0,grade_1,grade_2,grade_3,grade_4,grade_5,sex_セ,sex_牝,sex_牡,surface_ダ,surface_芝,weather_他,weather_小雨,weather_晴,weather_曇,weather_雨,weather_雪,preOOF_1.0,preOOF_10(降),preOOF_10.0,preOOF_11(降),preOOF_11.0,preOOF_12(降),preOOF_12.0,preOOF_13(降),preOOF_13.0,preOOF_14(降),preOOF_14.0,preOOF_15(降),preOOF_15.0,preOOF_16(降),preOOF_16.0,preOOF_17(降),preOOF_17.0,preOOF_18(降),preOOF_18.0,preOOF_2(降),preOOF_2.0,preOOF_3(降),preOOF_3.0,preOOF_4(降),preOOF_4.0,preOOF_5(降),preOOF_5.0,preOOF_6(降),preOOF_6.0,preOOF_7(降),preOOF_7.0,preOOF_8(降),preOOF_8.0,preOOF_9(降),preOOF_9.0,preOOF_中,preOOF_取,preOOF_失,preOOF_除,pre2OOF_1.0,pre2OOF_10(降),pre2OOF_10.0,pre2OOF_11(降),pre2OOF_11.0,pre2OOF_12(降),pre2OOF_12.0,pre2OOF_13(降),pre2OOF_13.0,pre2OOF_14(降),pre2OOF_14.0,pre2OOF_15(降),pre2OOF_15.0,pre2OOF_16(降),pre2OOF_16.0,pre2OOF_17(降),pre2OOF_17.0,pre2OOF_18(降),pre2OOF_18.0,pre2OOF_2(降),pre2OOF_2.0,pre2OOF_3(降),pre2OOF_3.0,pre2OOF_4(降),pre2OOF_4.0,pre2OOF_5(降),pre2OOF_5.0,pre2OOF_6(降),pre2OOF_6.0,pre2OOF_7(降),pre2OOF_7.0,pre2OOF_8(降),pre2OOF_8.0,pre2OOF_9(降),pre2OOF_9.0,pre2OOF_中,pre2OOF_取,pre2OOF_失,pre2OOF_除,month_1.0,month_2.0,month_3.0,month_4.0,month_5.0,month_6.0,month_7.0,month_8.0,month_9.0,month_10.0,month_11.0,month_12.0,ridingStrongJockey_0.0,ridingStrongJockey_1.0,course_他,course_右,course_外,course_左,course_直線,placeCode_中京,placeCode_中山,placeCode_京都,placeCode_函館,placeCode_小倉,placeCode_新潟,placeCode_札幌,placeCode_東京,placeCode_福島,placeCode_阪神,headCount_5.0,headCount_6.0,headCount_7.0,headCount_8.0,headCount_9.0,headCount_10.0,headCount_11.0,headCount_12.0,headCount_13.0,headCount_14.0,headCount_15.0,headCount_16.0,headCount_17.0,headCount_18.0,preHeadCount_5.0,preHeadCount_6.0,preHeadCount_7.0,preHeadCount_8.0,preHeadCount_9.0,preHeadCount_10.0,preHeadCount_11.0,preHeadCount_12.0,preHeadCount_13.0,preHeadCount_14.0,preHeadCount_15.0,preHeadCount_16.0,preHeadCount_17.0,preHeadCount_18.0,surfaceChanged_0.0,surfaceChanged_1.0,gradeChanged_-4.0,gradeChanged_-3.0,gradeChanged_-2.0,gradeChanged_-1.0,gradeChanged_0.0,gradeChanged_1.0,gradeChanged_2.0,gradeChanged_3.0,gradeChanged_4.0,gradeChanged_5.0,femaleOnly_0.0,femaleOnly_1.0,popularity_1.0,popularity_2.0,popularity_3.0,popularity_4.0,popularity_5.0,popularity_6.0,popularity_7.0,popularity_8.0,popularity_9.0,popularity_10.0,popularity_11.0,popularity_12.0,popularity_13.0,popularity_14.0,popularity_15.0,popularity_16.0,popularity_17.0,popularity_18.0,j-hweight,stable_地,stable_外,stable_東,stable_西,supPer,winRunTimes,preRunningStyle,pre2RunningStyle,order_was_2,order_was_3,not123times,changeDist,pre_last_phase_order
0,1,2006105662,1102,353,2.0,4.0,1200.0,0.0,0.0,452.0,0.0,0.0,0.0,54.0,0.0,0.0,0.0,36.277849,0.0,5,54.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.238938,0,0,1,0,0.114203,0.0,-1.0,-1.0,0,0,0.0,0.0,0.0
1,1,2006105747,635,429,2.0,0.0,1200.0,0.0,0.0,440.0,0.0,0.0,0.0,54.0,0.0,0.0,0.0,36.277849,0.0,6,54.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,

In [79]:
AUC(clf,X_test_std,y_test)

0.8593492110954501


In [179]:
AUC_(clf,X_test_std,y_test)

0.5358619275677955


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [166]:
importance_df = pd.DataFrame()
importance_df["feature"] = list(X_train_std.columns)
importance_df["importance"] = clf.feature_importances_
importance_df.sort_values('importance',ascending = False)

,feature,importance
2,changeDist,5095
89,owinper,5077
8,dhweight,5068
158,pre2RunningStyle,4966
216,pre_last_phase_order,4943
215,preRunningStyle,4873
227,supPer,4857
173,preLastPhase,4849
233,twinper,4679
1,basis_weight,4595


In [167]:
def forest_fit(X_train_std, y_train):
    from sklearn.ensemble import RandomForestClassifier
    forest = RandomForestClassifier(criterion='gini', n_estimators=30, random_state=1,n_jobs=2)
    forest.fit(X_train_std,y_train)
    return forest

In [168]:
forest=forest_fit(X_train_std,y_train)
AUC(forest,X_test_std, y_test)

0.7802704136785457


In [698]:
train_df.head()

,race_id,horse_number,grade,order_of_finish,horse_id,jockey_id,trainer_id,age,dhweight,disRoc,distance,enterTimes,eps,hweight,jwinper,odds,owinper,sex,surface,twinper,weather,weight,winRun,jEps,preOOF,pre2OOF,month,ridingStrongJockey,runningStyle,preLateStart,preLastPhase,lateStartPer,course,placeCode,headCount,preHeadCount,surfaceChanged,gradeChanged,preMargin,femaleOnly,frame_number,basis_weight,finishing_time,length,speed_figure,pass,last_phase,popularity,stable,earning_money,race_start,race_number,surface_score,date
0,1,9,4,1.0,2006105662,1102,353,2.0,4.0,NaN,1200.0,0.0,NaN,452.0,NaN,7.0,NaN,牝,芝,NaN,晴,54.0,NaN,NaN,NaN,NaN,11.0,NaN,0.000000,0.0,NaN,NaN,右,福島,16.0,NaN,NaN,NaN,NaN,1.0,5,54.0,1:10.2,NaN,NaN,1,36.3,4.0,東,500.0,10:00,1,NaN,2008-11-01
1,1,11,4,2.0,2006105747,635,429,2.0,0.0,NaN,1200.0,0.0,NaN,440.0,NaN,4.1,NaN,牝,芝,NaN,晴,54.0,NaN,NaN,NaN,NaN,11.0,NaN,0.266667,0.0,NaN,NaN,右,福島,16.0,NaN,NaN,NaN,NaN,1.0,6,54.0,1:10.3,1/2,NaN,5,36.0,2.0,西,200.0,10:00,1,NaN,2008-11-01
2,1,1,4,3.0,2006102910,1122,1076,2.0,-6.0,NaN,1200.0,0.0,NaN,428.0,NaN,2.4,NaN,牝,芝,NaN,晴,53.0,NaN,NaN,NaN,NaN,11.0,NaN,0.066667,0.0,NaN,NaN,右,福島,16.0,NaN,NaN,NaN,NaN,1.0,1,53.0,1:10.4,1/2,NaN,2,36.1,1.0,東,130.0,10:00,1,NaN,2008-11-01
3,1,7,4,4.0,2006100792,679,106,2.0,-10.0,NaN,1200.0,0.0,NaN,460.0,NaN,9.7,NaN,牝,芝,NaN,晴,54.0,NaN,NaN,NaN,NaN,11.0,NaN,0.066667,0.0,NaN,NaN,右,福島,16.0,NaN,NaN,NaN,NaN,1.0,4,54.0,1:10.4,クビ,NaN,2,36.3,5.0,東,75.0,10:00,1,NaN,2008-11-01
4,1,14,4,5.0,2006102658,1117,1067,2.0,0.0,NaN,1200.0,0.0,NaN,420.0,NaN,5.6,NaN,牝,芝,NaN,晴,51.0,NaN,NaN,NaN,NaN,11.0,NaN,0.333333,0.0,NaN,NaN,右,福島,16.0,NaN,NaN,NaN,NaN,1.0,7,51.0,1:10.5,クビ,NaN,6,36.0,3.0,東,50.0,10:00,1,NaN,2008-11-01


# 期待値計算

In [104]:
X_test_std.shape

(69667, 243)

In [107]:
df.isnull().sum()

age                       0
basis_weight              0
changeDist                0
course_他                  0
course_右                  0
course_外                  0
course_左                  0
course_直線                 0
dhweight                  0
distance                  0
enterTimes                0
eps                       0
femaleOnly_0.0            0
femaleOnly_1.0            0
frame_number              0
gradeChanged_-1.0         0
gradeChanged_-2.0         0
gradeChanged_-3.0         0
gradeChanged_-4.0         0
gradeChanged_0.0          0
gradeChanged_1.0          0
gradeChanged_2.0          0
gradeChanged_3.0          0
gradeChanged_4.0          0
gradeChanged_5.0          0
grade_0                   0
grade_0_fukusho_sum       0
grade_1                   0
grade_1_fukusho_sum       0
grade_2                   0
grade_2_fukusho_sum       0
grade_3                   0
grade_3_fukusho_sum       0
grade_4                   0
grade_4_fukusho_sum       0
grade_5             

In [114]:
train_df__ = train_df.dropna(subset=['order_of_finish'])
train_df__30000 = train_df__[train_df__['race_id']>30000]

In [115]:
train_df__30000.shape

(69667, 54)

In [116]:
X_test_std.shape

(69667, 247)

In [ ]:
train_df__ = train_df.dropna(subset=['order_of_finish'])
train_df__30000 = train_df__[train_df__['race_id']>30000]
pred=pd.DataFrame()
pred['race_id'] = train_df__30000['race_id']
pred['horse_id'] = train_df__30000['horse_id']
pred['horse_number'] = train_df__30000['horse_number']
pred['popularity']=train_df__30000['popularity']
pred['order_of_finish']=train_df__30000['order_of_finish']
pred['pred_prob']= clf.predict_proba(X_test_std)[:,1]
pred['grade']=train_df__30000['grade']
pred.head(100)

In [786]:
sorted_ = pred[pred['race_id']==26180].sort_values('pred_prob' ,ascending = False)
sorted_

,race_id,horse_id,horse_number,popularity,order_of_finish,pred_prob,grade
379274,26180,2013105503,6,1.0,5.0,0.624910,5
379272,26180,2013106097,12,4.0,3.0,0.102369,5
379270,26180,2013110124,5,2.0,1.0,0.094176,5
379271,26180,2013105833,4,3.0,2.0,0.079514,5
379283,26180,2013102089,8,5.0,14.0,0.078425,5
379280,26180,2013106388,13,7.0,11.0,0.022551,5
379277,26180,2013101455,14,8.0,8.0,0.020089,5
379275,26180,2013103191,2,6.0,6.0,0.018610,5
379278,26180,2013103563,3,10.0,9.0,0.013701,5
379284,26180,2013101587,15,9.0,15.0,0.007344,5


In [765]:
this_payoff = payoff[payoff['race_id']==29032]
this_payoff

,race_id,ticket_type,horse_number,payoff,popularity
348223,29032,0,3,680.0,4
348224,29032,1,3,270.0,4
348225,29032,1,2,330.0,7
348226,29032,1,1,290.0,6
348227,29032,2,2 - 3,3660.0,24
348228,29032,3,2 - 3,3480.0,18
348229,29032,4,2 - 3,1580.0,27
348230,29032,4,1 - 3,980.0,9
348231,29032,4,1 - 2,1540.0,26
348232,29032,5,3 → 2,7560.0,43


In [135]:
win_horse_list[0:2]

['15', '8']

In [ ]:
pred_ = pred

payoff_0 = 0
payoff_1 = 0
payoff_3 = 0
payoff_4 = 0
payoff_5 = 0
payoff_6 = 0
payoff_7 = 0
for race in pred_.race_id.unique():
    race_count = len(pred_.race_id.unique())
    
    sorted_ = pred_[pred_['race_id']==race].sort_values('pred_prob' ,ascending = False)
    win_horse_list = [str(sorted_['horse_number'][i:i+1].values[0]) for i in range(5)]
    
    this_payoff = payoff[payoff['race_id']==race]
    
    #単勝0
    p_0 = 0
    if this_payoff[payoff['ticket_type']==0]['horse_number'].values[0] in win_horse_list[0:1] : #1頭買う
        p_0 = this_payoff[this_payoff['ticket_type']==0]['payoff'].values[0]
        payoff_0 += p_0
        
    #複勝1
    p_1 = 0
    fukusho_num = this_payoff[this_payoff['ticket_type']==1]['horse_number'].values
    for i in range(len(fukusho_num)):
        if fukusho_num[i] in win_horse_list[0:3] : #3頭買う
            p_1 += this_payoff[this_payoff['ticket_type']==1]['payoff'].values[i]
            payoff_1 += p_1
    #馬連3
    umaren_num = this_payoff[this_payoff['ticket_type']==3]['horse_number'].values[0].replace(' ','').split('-')
    p_3 = 0
    if umaren_num[0]==win_horse_list[0] and umaren_num[1]==win_horse_list[1] :
        p_3 = this_payoff[this_payoff['ticket_type']==3]['payoff'].values[0]
        payoff_3 += p_3
    elif umaren_num[0]==win_horse_list[1] and umaren_num[1]==win_horse_list[0]:
        p_3 = this_payoff[this_payoff['ticket_type']==3]['payoff'].values[0]
        payoff_3 += p_3
    #ワイド4
    p_4 = 0
    for i in range(3):
        wide_num = this_payoff[this_payoff['ticket_type']==4]['horse_number'].values[i].replace(' ','').split('-')
        if wide_num[0] in win_horse_list[0:3] and wide_num[1] in win_horse_list[0:3] :
            p_4 += this_payoff[this_payoff['ticket_type']==4]['payoff'].values[i]
            payoff_4 += p_4
    #馬単5
    p_5 = 0
    umatan_num = this_payoff[this_payoff['ticket_type']==5]['horse_number'].values[0].replace(' ','').split('→')
    if umatan_num[0]==win_horse_list[0] and umatan_num[1]==win_horse_list[1] :
        p_5 = this_payoff[this_payoff['ticket_type']==5]['payoff'].values[0]
        payoff_5 += p_5
    #３連複６
    p_6 = 0
    sanrenpuku_num = this_payoff[this_payoff['ticket_type']==6]['horse_number'].values[0].replace(' ','').split('-')
    if sanrenpuku_num[0] in win_horse_list[0:3] and sanrenpuku_num[1] in win_horse_list[0:3] and sanrenpuku_num[2] in win_horse_list[0:3]:
        p_6 = this_payoff[this_payoff['ticket_type']==6]['payoff'].values[0]
        payoff_6 += p_6
    #３連単7
    p_7 = 0
    sanrentan_num = this_payoff[this_payoff['ticket_type']==7]['horse_number'].values[0].replace(' ','').split('→')
    if sanrentan_num[0]==win_horse_list[0] and sanrentan_num[1]==win_horse_list[1]  and sanrentan_num[2]==win_horse_list[2] :
        p_7 = this_payoff[this_payoff['ticket_type']==7]['payoff'].values[0]
        payoff_7 += p_7
    
    print(race,'0:',p_0,'1:',p_1,'3:',p_3,'4:',p_4,'5:',p_5,'6:',p_6,'7:',p_7)

av_0 = payoff_0/race_count
av_1 = payoff_1/race_count
av_3 = payoff_3/race_count
av_4 = payoff_4/race_count
av_5 = payoff_5/race_count
av_6 = payoff_6/race_count
av_7 = payoff_7/race_count

print(av_0 ,av_1 ,av_3,av_4 ,av_5 ,av_6 ,av_7)

In [ ]:
print(
'単勝：',av_0/100,'\n',
'複勝：' ,av_1/300,'\n',
'馬連：',av_3 / 200,'\n',
'ワイド：', av_4 /300,'\n',
'馬単：', av_5/100,'\n',
'３連複：', av_6/ 100,'\n',
'3連単：', av_7 /100)

In [141]:
pred

,race_id,horse_id,horse_number,popularity,order_of_finish,pred_prob,grade
434508,30001,2015101597,13,1.0,1.0,0.539133,4
434509,30001,2015100216,9,2.0,2.0,0.107992,4
434510,30001,2015100811,12,4.0,3.0,0.068930,4
434511,30001,2015103384,2,6.0,4.0,0.033186,4
434512,30001,2015105725,1,5.0,5.0,0.022538,4
434513,30001,2015102684,3,12.0,6.0,0.010008,4
434514,30001,2015101066,14,8.0,7.0,0.036291,4
434515,30001,2015100295,4,3.0,8.0,0.069471,4
434516,30001,2015101876,5,7.0,9.0,0.032177,4
434517,30001,2015106429,7,9.0,10.0,0.023249,4


# 予測

In [130]:
test_df = pd.read_csv('test_df201807040411.csv')
test_df['date'] = '2018-12-08'
test_df['month']=12

In [129]:
def PreProTest(PreProData , test_df):
    this_race_id  =  test_df['race_id'][0]
    data = test_df
    
    #odds 無限でfillna
    data['odds']= data['odds'].fillna(np.inf)
    
    #脚質を計算、追加、元の脚質情報は削除　（通過順位−１）／（頭数−１）
#    data['pass'] = data['pass'].fillna(-1)
 #   data['pass'] = data['pass'].map(passOrd)
 #   data['runningStyle']=(data['pass']-1)/(data['headCount']-1)
    
    #preOOF,pre2OOFの数字を整数に
    data['preOOF']= data['preOOF'].map(strFloat_to_strInt)
    data['pre2OOF']= data['pre2OOF'].map(strFloat_to_strInt)
    
    #fillna_0
    fillna_0_cols = ['ridingStrongJockey', 'eps', 'jwinper','owinper','twinper','winRun', 'jEps','runningStyle','lateStartPer']
    data = fillna_0(data, fillna_0_cols)

    #fillna_mean
    fillna_mean_cols = ['dhweight', 'hweight', 'preLastPhase']
    data = fillna_mean(data ,fillna_mean_cols)
    
    #odds 無限でfillna
    data['odds']= data['odds'].fillna(np.inf) 
    
    #popularity
    data['popularity']  = float(-1.0)
    data['popularity'] = data['popularity'].astype(float)
    for i in range(len(test_df)):
        i_ninki_horse_id = data.sort_values('odds', ascending = True)[i:i+1]['horse_id'].values[0]
        i_ninki_idx  = data[data['horse_id']==i_ninki_horse_id].index[0]
        data.at[i_ninki_idx, 'popularity'] = float(i+1)
    
    S=pd.get_dummies(data[["popularity"]].astype(object))
    data=pd.concat([data, S], axis=1)
    data=data.drop('popularity',axis=1)
    del S
    gc.collect()
    

    #j-hweight　　（騎手＋斤量）／馬の体重
#    data['j-hweight']=(data['basis_weight']+data['weight']) / data['hweight']

    #stable
#    Q=pd.get_dummies(data[["stable"]].astype(object))
#    data=pd.concat([data, Q], axis=1)
#    data=data.drop('stable',axis=1)
#    del Q
#    gc.collect()

    #支持率
    data['supPer'] = 0.788/(data['odds']-0.1)
    data=data.drop('odds',axis=1)

    #馬の１着数
    data['winRunTimes']=data['winRun']*data['enterTimes']

    #そのレースの脚質の平均
#    runStyle=pd.DataFrame(data.groupby('race_id',as_index=False)[['runningStyle']].mean())
#    runStyle = runStyle.rename(columns={'runningStyle': 'all_runnnigStyle'})
#    data = pd.merge(data, runStyle , on='race_id' ,how='left')
    
    #前走、前々走の脚質
    data['preRunningStyle'] = -1
    data['pre2RunningStyle'] =-1
    data[['preRunningStyle','pre2RunningStyle']] = data[['preRunningStyle','pre2RunningStyle']].astype(float)
    for n in range(len(test_df)):
        n_horse_id=test_df['horse_id'].loc[n]
        pre = -1
        pre2 = -1
        if data['enterTimes'][n] ==1:
            pre = PreProData[PreProData['horse_id']==n_horse_id].sort_values('date',ascending=False)[0:1]['runningStyle'].values[0]
        elif data['enterTimes'][n] >=2:
            pre = PreProData[PreProData['horse_id']==n_horse_id].sort_values('date',ascending=False)[0:1]['runningStyle'].values[0]
            pre2 = PreProData[PreProData['horse_id']==n_horse_id].sort_values('date',ascending=False)[0:1]['preRunningStyle'].values[0]
        data.at[n,'preRunningStyle'] = pre
        data.at[n,'pre2RunningStyle'] = pre2
        
    #2着数、３着数
    data['order_was_2']=0
    data['order_was_3']=0
    for n in range(len(test_df)):
        n_horse_id=test_df['horse_id'].loc[n]
        order_was_2 = 0
        order_was_3 = 0
        if data['enterTimes'][n] >= 1:
            order_was_2 = PreProData[PreProData['horse_id']==n_horse_id].sort_values('date',ascending=False)[0:1]['order_was_2'].values[0]
            order_was_3 = PreProData[PreProData['horse_id']==n_horse_id].sort_values('date',ascending=False)[0:1]['order_was_3'].values[0]
            if PreProData[PreProData['horse_id']==n_horse_id].sort_values('date',ascending=False)[0:1]['order_of_finish'].values[0] == 2:
                order_was_2 += 1
            elif PreProData[PreProData['horse_id']==n_horse_id].sort_values('date',ascending=False)[0:1]['order_of_finish'].values[0] == 3:
                order_was_3 += 1
        data.at[n,'order_was_2'] = order_was_2
        data.at[n,'order_was_3'] = order_was_3
        
    #1~3着でない回数
    data['not123times'] = data['enterTimes']-data['winRunTimes']-data['order_was_2']-data['order_was_3']
    
    
    #距離変化量 changedist
    data['changeDist'] = 0
    data['changeDist'] = data['changeDist'].astype(float)
    for n in range(len(test_df)):
        n_horse_id=test_df['horse_id'].loc[n]
        changeDist = 0
        if data['enterTimes'][n] >= 1:
            preDist = PreProData[PreProData['horse_id']==n_horse_id].sort_values('date',ascending=False)[0:1]['distance'].values[0]
            changeDist = data.at[n,'distance'] - preDist
        data.at[n,'changeDist'] = changeDist
        
    #前回の３ハロン順位
    data['pre_last_phase_order'] = -1
    data['pre_last_phase_order'] = data['pre_last_phase_order'].astype(float)
    for n in range(len(test_df)):
        n_horse_id=test_df['horse_id'].loc[n]
        preLastPhaseOrder = -1
        if data['enterTimes'][n] >= 1:
            preLastPhaseOrder = PreProData[PreProData['horse_id']==n_horse_id].sort_values('date',ascending=False)[0:1]['last_phase_order'].values[0]
        data.at[n,'pre_last_phase_order'] = float(preLastPhaseOrder)
    
    
    
    #ランク別の複勝数  
    data['grade_0_fukusho_sum'] = 0
    data['grade_1_fukusho_sum'] = 0
    data['grade_2_fukusho_sum'] = 0
    data['grade_3_fukusho_sum'] = 0
    data['grade_4_fukusho_sum'] = 0
    data['grade_5_fukusho_sum'] = 0
    data['this_grade_fukusho_sum']  = 0
    for n in range(len(test_df)):
        n_horse_id=test_df['horse_id'].loc[n]
        last_race = PreProData[PreProData['horse_id']==n_horse_id].sort_values('date',ascending=False)[0:1]
        data.at[n,'grade_0_fukusho_sum'] = last_race['grade_0_fukusho_sum'].values[0]
        data.at[n,'grade_1_fukusho_sum'] = last_race['grade_1_fukusho_sum'].values[0]
        data.at[n,'grade_2_fukusho_sum'] = last_race['grade_2_fukusho_sum'].values[0]
        data.at[n,'grade_3_fukusho_sum'] = last_race['grade_3_fukusho_sum'].values[0]
        data.at[n,'grade_4_fukusho_sum'] = last_race['grade_4_fukusho_sum'].values[0]
        data.at[n,'grade_5_fukusho_sum'] = last_race['grade_5_fukusho_sum'].values[0]
        if last_race['order_of_finish'].values[0]<=3:
            if last_race['grade'].values[0] ==0 :
                data.at[n,'grade_0_fukusho_sum'] += 1
            elif last_race['grade'].values[0] ==1 :
                data.at[n,'grade_1_fukusho_sum'] += 1
            elif last_race['grade'].values[0] ==2 :
                data.at[n,'grade_2_fukusho_sum'] += 1
            elif last_race['grade'].values[0] ==3 :
                data.at[n,'grade_3_fukusho_sum'] += 1
            elif last_race['grade'].values[0] ==4 :
                data.at[n,'grade_4_fukusho_sum'] += 1
            elif last_race['grade'].values[0] ==5 :
                data.at[n,'grade_5_fukusho_sum'] += 1

        if data.at[n,'grade'] == 0:
            this_grade_fukusho_sum = data.at[n, 'grade_0_fukusho_sum']
        elif data.at[n,'grade'] == 1:
            this_grade_fukusho_sum = data.at[n, 'grade_0_fukusho_sum'] + data.at[n, 'grade_1_fukusho_sum']
        elif data.at[n,'grade'] == 2:
            this_grade_fukusho_sum = data.at[n, 'grade_0_fukusho_sum'] + data.at[n, 'grade_1_fukusho_sum'] +data.at[n,
                                                                                                                    'grade_2_fukusho_sum']
        elif data.at[n,'grade'] == 3:
            this_grade_fukusho_sum = data.at[n, 'grade_0_fukusho_sum'] + data.at[n, 'grade_1_fukusho_sum'] +data.at[n, 
                                                                                                                    'grade_2_fukusho_sum'] + data.at[n, 'grade_3_fukusho_sum']
        elif data.at[n,'grade'] == 4:
            this_grade_fukusho_sum = data.at[n, 'grade_0_fukusho_sum'] + data.at[n, 'grade_1_fukusho_sum'] +data.at[n,
                                                                                                                    'grade_2_fukusho_sum'] + data.at[n, 'grade_3_fukusho_sum'] + data.at[n, 'grade_4_fukusho_sum']
        elif data.at[n,'grade'] == 5:
            this_grade_fukusho_sum = data.at[n, 'grade_0_fukusho_sum'] + data.at[n, 'grade_1_fukusho_sum'] +data.at[n, 'grade_2_fukusho_sum'] + data.at[n, 'grade_3_fukusho_sum'] + data.at[n, 'grade_4_fukusho_sum'] + data.at[n, 'grade_5_fukusho_sum']
        data.at[n, 'this_grade_fukusho_sum'] = this_grade_fukusho_sum

    
    
    
    #ダミー変数化
    dammy_cols = ['horse_number','grade','sex','surface','weather','preOOF',
                     'pre2OOF' ,'month', 'ridingStrongJockey' , 'course' ,'placeCode' ,'headCount' ,
                     'preHeadCount' ,'surfaceChanged','gradeChanged','femaleOnly','popularity','stable']
    data = dammy(data, dammy_cols)
    
   
    return data

In [131]:
test=PreProTest(data_ ,test_df)

KeyError: 'last_phase_order'

In [515]:
X_train.head()

,age,dhweight,distance,enterTimes,eps,hweight,jwinper,owinper,twinper,weight,winRun,jEps,preLateStart,preLastPhase,lateStartPer,frame_number,basis_weight,race_number,horse_number_1,horse_number_2,horse_number_3,horse_number_4,horse_number_5,horse_number_6,horse_number_7,horse_number_8,horse_number_9,horse_number_10,horse_number_11,horse_number_12,horse_number_13,horse_number_14,horse_number_15,horse_number_16,horse_number_17,horse_number_18,grade_0,grade_1,grade_2,grade_3,grade_4,grade_5,sex_セ,sex_牝,sex_牡,surface_ダ,surface_芝,weather_他,weather_小雨,weather_晴,weather_曇,weather_雨,weather_雪,preOOF_1,preOOF_10,preOOF_10(降),preOOF_11,preOOF_11(降),preOOF_12,preOOF_12(降),preOOF_13,preOOF_13(降),preOOF_14,preOOF_14(降),preOOF_15,preOOF_15(降),preOOF_16,preOOF_16(降),preOOF_17,preOOF_17(降),preOOF_18,preOOF_18(降),preOOF_2,preOOF_2(降),preOOF_3,preOOF_3(降),preOOF_4,preOOF_4(降),preOOF_5,preOOF_5(降),preOOF_6,preOOF_6(降),preOOF_7,preOOF_7(降),preOOF_8,preOOF_8(降),preOOF_9,preOOF_9(降),preOOF_nan,preOOF_中,preOOF_取,preOOF_失,preOOF_除,pre2OOF_1,pre2OOF_10,pre2OOF_10(降),pre2OOF_11,pre2OOF_11(降),pre2OOF_12,pre2OOF_12(降),pre2OOF_13,pre2OOF_13(降),pre2OOF_14,pre2OOF_14(降),pre2OOF_15,pre2OOF_15(降),pre2OOF_16,pre2OOF_16(降),pre2OOF_17,pre2OOF_17(降),pre2OOF_18,pre2OOF_18(降),pre2OOF_2,pre2OOF_2(降),pre2OOF_3,pre2OOF_3(降),pre2OOF_4,pre2OOF_4(降),pre2OOF_5,pre2OOF_5(降),pre2OOF_6,pre2OOF_6(降),pre2OOF_7,pre2OOF_7(降),pre2OOF_8,pre2OOF_8(降),pre2OOF_9,pre2OOF_9(降),pre2OOF_nan,pre2OOF_中,pre2OOF_取,pre2OOF_失,pre2OOF_除,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,ridingStrongJockey_0.0,ridingStrongJockey_1.0,course_他,course_右,course_外,course_左,course_直線,placeCode_中京,placeCode_中山,placeCode_京都,placeCode_函館,placeCode_小倉,placeCode_新潟,placeCode_札幌,placeCode_東京,placeCode_福島,placeCode_阪神,headCount_5.0,headCount_6.0,headCount_7.0,headCount_8.0,headCount_9.0,headCount_10.0,headCount_11.0,headCount_12.0,headCount_13.0,headCount_14.0,headCount_15.0,headCount_16.0,headCount_17.0,headCount_18.0,preHeadCount_5.0,preHeadCount_6.0,preHeadCount_7.0,preHeadCount_8.0,preHeadCount_9.0,preHeadCount_10.0,preHeadCount_11.0,preHeadCount_12.0,preHeadCount_13.0,preHeadCount_14.0,preHeadCount_15.0,preHeadCount_16.0,preHeadCount_17.0,preHeadCount_18.0,surfaceChanged_0.0,surfaceChanged_1.0,gradeChanged_-4.0,gradeChanged_-3.0,gradeChanged_-2.0,gradeChanged_-1.0,gradeChanged_0.0,gradeChanged_1.0,gradeChanged_2.0,gradeChanged_3.0,gradeChanged_4.0,gradeChanged_5.0,femaleOnly_0.0,femaleOnly_1.0,popularity_1.0,popularity_2.0,popularity_3.0,popularity_4.0,popularity_5.0,popularity_6.0,popularity_7.0,popularity_8.0,popularity_9.0,popularity_10.0,popularity_11.0,popularity_12.0,popularity_13.0,popularity_14.0,popularity_15.0,popularity_16.0,popularity_17.0,popularity_18.0,j-hweight,stable_地,stable_外,stable_東,stable_西,supPer,winRunTimes,preRunningStyle,pre2RunningStyle,order_was_2,order_was_3,not123times,changeDist,pre_last_phase_order
0,2.0,4.0,1200.0,0.0,0.0,452.0,0.0,0.0,0.0,54.0,0.0,0.0,0.0,36.277849,0.0,5,54.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.238938,0,0,1,0,0.114203,0.0,-1.0,-1.0,0,0,0.0,0.0,0.0
1,2.0,0.0,1200.0,0.0,0.0,440.0,0.0,0.0,0.0,54.0,0.0,0.0,0.0,36.277849,0.0,6,54.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.245455

In [505]:
test.shape

(14, 111)

In [653]:
test=test.drop({'race_id','horse_id',
            'jockey_id','trainer_id','preMargin','finishing_time','length','speed_figure',
           'last_phase','race_start','surface_score',
            'earning_money','date','last_phase_order','runningStyle','pass','frame_number'}, axis=1)

KeyError: "['finishing_time' 'length' 'speed_figure' 'frame_number' 'earning_money'\n 'race_start' 'surface_score' 'last_phase_order' 'pass' 'last_phase'] not found in axis"

In [539]:
test.shape

(14, 111)

In [654]:
test = test.drop({'basis_weight', 'preLateStart', 'race_number', 'stable'}, axis=1)

KeyError: "['stable' 'basis_weight' 'preLateStart' 'race_number'] not found in axis"

In [655]:
#testカラムで必要なものを追加、不要なものを削除

src_set = set(test.columns)
tag_set = set(X_train.columns)

if list(set(tag_set) -set(src_set & tag_set)) != []:
    train_margin = list(set(tag_set) -set(src_set & tag_set))
    for i in range(len(train_margin)):
        test[train_margin[i]]=np.zeros(test.shape[0])
        
if list(set(src_set) -set(src_set & tag_set)) != [] :
    test_margin = list(set(src_set) -set(src_set & tag_set))
    test=test.drop(columns =test_margin)

test=test.sort_index(axis=1)

In [656]:
test = test.sort_index(axis=1)
#transformで、fitで推定した値を元に標準化
test_std = sc.transform(test)

In [657]:
test.head()

,age,basis_weight,changeDist,course_他,course_右,course_外,course_左,course_直線,dhweight,distance,enterTimes,eps,femaleOnly_0.0,femaleOnly_1.0,frame_number,gradeChanged_-1.0,gradeChanged_-2.0,gradeChanged_-3.0,gradeChanged_-4.0,gradeChanged_0.0,gradeChanged_1.0,gradeChanged_2.0,gradeChanged_3.0,gradeChanged_4.0,gradeChanged_5.0,grade_0,grade_1,grade_2,grade_3,grade_4,grade_5,headCount_10.0,headCount_11.0,headCount_12.0,headCount_13.0,headCount_14.0,headCount_15.0,headCount_16.0,headCount_17.0,headCount_18.0,headCount_5.0,headCount_6.0,headCount_7.0,headCount_8.0,headCount_9.0,horse_number_1,horse_number_10,horse_number_11,horse_number_12,horse_number_13,horse_number_14,horse_number_15,horse_number_16,horse_number_17,horse_number_18,horse_number_2,horse_number_3,horse_number_4,horse_number_5,horse_number_6,horse_number_7,horse_number_8,horse_number_9,hweight,j-hweight,jEps,jwinper,lateStartPer,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,not123times,order_was_2,order_was_3,owinper,placeCode_中京,placeCode_中山,placeCode_京都,placeCode_函館,placeCode_小倉,placeCode_新潟,placeCode_札幌,placeCode_東京,placeCode_福島,placeCode_阪神,popularity_1.0,popularity_10.0,popularity_11.0,popularity_12.0,popularity_13.0,popularity_14.0,popularity_15.0,popularity_16.0,popularity_17.0,popularity_18.0,popularity_2.0,popularity_3.0,popularity_4.0,popularity_5.0,popularity_6.0,popularity_7.0,popularity_8.0,popularity_9.0,pre2OOF_1,pre2OOF_10,pre2OOF_10(降),pre2OOF_11,pre2OOF_11(降),pre2OOF_12,pre2OOF_12(降),pre2OOF_13,pre2OOF_13(降),pre2OOF_14,pre2OOF_14(降),pre2OOF_15,pre2OOF_15(降),pre2OOF_16,pre2OOF_16(降),pre2OOF_17,pre2OOF_17(降),pre2OOF_18,pre2OOF_18(降),pre2OOF_2,pre2OOF_2(降),pre2OOF_3,pre2OOF_3(降),pre2OOF_4,pre2OOF_4(降),pre2OOF_5,pre2OOF_5(降),pre2OOF_6,pre2OOF_6(降),pre2OOF_7,pre2OOF_7(降),pre2OOF_8,pre2OOF_8(降),pre2OOF_9,pre2OOF_9(降),pre2OOF_nan,pre2OOF_中,pre2OOF_取,pre2OOF_失,pre2OOF_除,pre2RunningStyle,preHeadCount_10.0,preHeadCount_11.0,preHeadCount_12.0,preHeadCount_13.0,preHeadCount_14.0,preHeadCount_15.0,preHeadCount_16.0,preHeadCount_17.0,preHeadCount_18.0,preHeadCount_5.0,preHeadCount_6.0,preHeadCount_7.0,preHeadCount_8.0,preHeadCount_9.0,preLastPhase,preLateStart,preOOF_1,preOOF_10,preOOF_10(降),preOOF_11,preOOF_11(降),preOOF_12,preOOF_12(降),preOOF_13,preOOF_13(降),preOOF_14,preOOF_14(降),preOOF_15,preOOF_15(降),preOOF_16,preOOF_16(降),preOOF_17,preOOF_17(降),preOOF_18,preOOF_18(降),preOOF_2,preOOF_2(降),preOOF_3,preOOF_3(降),preOOF_4,preOOF_4(降),preOOF_5,preOOF_5(降),preOOF_6,preOOF_6(降),preOOF_7,preOOF_7(降),preOOF_8,preOOF_8(降),preOOF_9,preOOF_9(降),preOOF_nan,preOOF_中,preOOF_取,preOOF_失,preOOF_除,preRunningStyle,pre_last_phase_order,race_number,ridingStrongJockey_0.0,ridingStrongJockey_1.0,sex_セ,sex_牝,sex_牡,stable_地,stable_外,stable_東,stable_西,supPer,surfaceChanged_0.0,surfaceChanged_1.0,surface_ダ,surface_芝,twinper,weather_他,weather_小雨,weather_晴,weather_曇,weather_雨,weather_雪,weight,winRun,winRunTimes
0,5.0,0.0,200.0,0.0,0.0,0.0,1,0.0,8,2200.0,31.0,59.658065,1,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,486,0.0,99.392810,0.056,0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,1,4,0.065,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.444444,0,0.0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0,34.5,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,10.0,0.0,0,1,0.0,0,1,0.0,0.0,0.0,0.0,0.042366,1,0.0,0.0,1,0.064,0.0,0.0,1,0.0,0.0,0.0,57.0,0.000000,0.0
1,4.0,0.0,500.0,0.0,0.0,0.0,1,0.0,2,2200.0,18.0,86.111111,1,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.

In [658]:
ans = pd.DataFrame(clf.predict_proba(test_std)[:,1])
ans.columns=["prob"]
ans = ans.reset_index().rename(columns={'index': 'h_num'})
ans['h_num'] += 1
ans

,h_num,prob
0,1,0.030476
1,2,0.001187
2,3,0.013124
3,4,0.064074
4,5,0.024478
5,6,0.004787
6,7,0.010677
7,8,0.018324
8,9,0.020160
9,10,0.027140


In [659]:
ans.sort_values('prob',ascending = False)

,h_num,prob
3,4,0.064074
0,1,0.030476
9,10,0.027140
4,5,0.024478
8,9,0.020160
7,8,0.018324
2,3,0.013124
6,7,0.010677
5,6,0.004787
11,12,0.003476
